In [29]:
from src.data.dataset import load_data
import torch
import numpy as np
import pickle
from tqdm import tqdm
from torchmetrics.classification import MultilabelJaccardIndex

In [3]:
x_to_c = torch.load('ConceptModel__Seed1/outputs/best_model_1.pth')
c_to_y = torch.load('IndependentModel_WithVal___Seed1/outputs/best_model_1.pth')

In [4]:
test_loader = load_data(pkl_paths=['CUB_processed/class_attr_data_10/test.pkl'],use_attr=True,no_img=False,batch_size=32)

In [60]:
cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
jac = MultilabelJaccardIndex(num_labels=112,average="weighted")
jac.cuda()

cosine_similarities = []
outputs_all = []
concepts_all = []

x_to_c.eval()
for images, labels, concepts in tqdm(test_loader):
    image_b = images.cuda(non_blocking=True)
    label = labels.cuda(non_blocking=True)
    concepts = torch.stack(concepts,dim=1).cuda(non_blocking=True)

    with torch.no_grad():
        outputs = x_to_c(image_b)
    
    outputs = torch.squeeze(torch.stack(outputs,dim=1))
    outputs = torch.nn.functional.sigmoid(outputs)
    
    outputs_all.append(outputs)
    concepts_all.append(concepts)
    
    cosine_similarities.append(cos(outputs,concepts))
    
cosine_similarities = torch.cat(cosine_similarities)
outputs_all = torch.cat(outputs_all)
concepts_all = torch.cat(concepts_all)

jaccard_similarity = jac(outputs_all,concepts_all)

100%|█████████████████████████████████████████████████████████████████████████████████| 182/182 [00:45<00:00,  4.00it/s]


In [61]:
print("Jaccard similarity:",jaccard_similarity.item())
print("Cosine similarity:",cosine_similarities.mean().item())

Jaccard similarity: 0.8566474914550781
Cosine similarity: 0.9317216277122498
